# 2.1 내장 시퀀스 개요

파이썬 표준 라이브러리는 C로 구현된 다음 시퀀스형을 제공한다.

* 컨테이너 시퀀스 (객체에 대한 참조를 담음)
    * 서로 다른 자료형의 항목들을 담을 수 있는 list, tuple, collections.deque 형
* 균일 시퀀스 (자신의 메모리 공간에 각 항목의 값을 담음 > 메모리를 더 적게 사용)
    * 단 하나의 자료형만 담을 수 있는 str, bytes, bytearray, memoryview, array.array 형
    
가변성에 따라 다음과 같이 분류할 수도 있다.

* 가변 시퀀스
    * list, bytearray, array.array, collections.deque, memoryview 형
* 불변 시퀀스
    * tuple, str, bytes 형

# 2.2 지능형 리스트와 제너레이터 표현식

지능형 리스트(리스트형의 경우)나 제너레이터 표현식(그 외 시퀀스의 경우)을 사용하면 가독성이 좋고 때로는 실행 속도도 빠른 코드를 만들 수 있다.

## 2.2.1 지능형 리스트와 가독성

다음 두 코드 중 어느 코드가 읽기 쉬운가?

In [1]:
symbols = "$*&!@"
codes = []
for symbol in symbols:
    codes.append(ord(symbol)) # ord : Return the Unicode code point for a one-character string.
print(codes)

[36, 42, 38, 33, 64]


In [2]:
symbols = "$*&!@"
codes = [ord(symbol) for symbol in symbols]
print(codes)

[36, 42, 38, 33, 64]


for 문은 다양한 일에 사용할 수 있다. 그러나 지능형 리스트는 오로지 새로운 리스트를 만드는 일만 한다. 

* 파이썬에서는 [], {}, () 안에서의 개행이 무시되므로 \ 를 사용하지 않고도 여러 줄에 걸쳐 리스트, 지능형 리스트, 제너레이터 표현식, 딕셔너리를 작성할 수 있다.

## 2.2.2 지능형 리스트와 map()/filter() 비교

지능형 리스트는 filter() 와 map() 함수를 이용해서도 구현할 수 있지만 가독성이 떨어진다.

In [5]:
symbols = "$*&!@"
test = [ord(s) for s in symbols if ord(s) > 40] # 지능형 리스트%
print(test)

[42, 64]


In [6]:
symbols = "$*&!@"
test = list(filter(lambda c: c>40, map(ord, symbols)))
print(test)

[42, 64]


In [10]:
map(ord, symbols), list(map(ord, symbols))

(<map at 0x7f817d76e6d8>, [36, 42, 38, 33, 64])

In [11]:
filter(lambda c: c>40, map(ord, symbols)), list(filter(lambda c: c>40, map(ord, symbols)))

(<filter at 0x7f817d76ee48>, [42, 64])

두 방법의 성능을 비교해보자.

In [17]:
import math

In [36]:
%%timeit
ans = [math.log(i) for i in range(1,100000000) if i>1000000]
print(len(ans))

98999999
98999999
98999999
98999999
98999999
98999999
98999999
98999999
12.4 s ± 121 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
ans = list(map(math.log, filter(lambda c: c>1000000, range(1,100000000))))
print(len(ans))

98999999
98999999
98999999
98999999
98999999
98999999
98999999
98999999
12.2 s ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


거의 차이가 없구만...

## 2.2.3 데카르트 곱

데카르트 곱은 두 개 이상의 리스트에 있는 모든 항목을 이용해서 만든 튜플로 구성된 리스트이며, 지능형 리스트를 통해서도 만들 수 있다.

In [39]:
colors = ['black', 'white']
sizes = list('SML')
tshirts = [(color, size) for size in sizes
                                  for color in colors]
print(tshirts)

[('black', 'S'), ('white', 'S'), ('black', 'M'), ('white', 'M'), ('black', 'L'), ('white', 'L')]


## 2.2.4 제너레이터 표현식

튜플, 배열 등의 시퀀스형을 초기화하려면 지능형 리스트를 사용할 수도 있지만, 다른 생성자에 전달할 리스트를 통째로 만들지 않고 반복자 프로토콜을 이용해서 항목을 하나씩 생성하는 제너레이터 표현식이 메모리를 더 적게 사용한다. 제너레이터 표현식은 대괄호 대신 괄호를 사용한다.

In [40]:
symbols = "$*&!@"
test = tuple(ord(s) for s in symbols if ord(s) > 40)
print(test)

(42, 64)


In [41]:
import array
test = array.array('I', (ord(symbol) for symbol in symbols))
print(test)

array('I', [36, 42, 38, 33, 64])


데카르트 곱을 만들기 위해 사용할 리스트에 각각 천 개의 항목이 들어있는 경우 제너레이터 표현식을 사용하면 단지 for 루프에 전달하기 위해 항목이 백만 개 들어 있는 리스트를 생성하는 일을 피할 수 있다.

In [54]:
colors = ['black', 'white']
sizes = list('SML')
for tshirt in ('{} {}'.format(color,size) for size in sizes for color in colors):
    print(tshirt)

black S
white S
black M
white M
black L
white L


# 2.3 튜플은 단순한 불변 리스트가 아니다.

튜플은 불변 리스트로 사용할 수도 있지만 필드명이 없는 레코드로 사용할 수도 있다.

## 2.3.1 레코드로서의 튜플

튜플의 각 항목은 레코드의 필드 하나를 의미하며 항목의 위치가 의미를 결정한다. 튜플을 단지 불변 리스트로 생각한다면 경우에 따라 항목의 크기와 순서가 중요하지 않을 수도 있다. 그러나 튜플을 필드의 집합으로 사용하는 경우에는 항목 수가 고정되어 있고 그 순서가 중요하다. 다음은 튜플을 레코드로 사용하는 예제이다.

In [56]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP','XDA205856')]

In [57]:
for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

BRA/CE342567
ESP/XDA205856
USA/31195855


In [58]:
for country, _ in traveler_ids:
    print(country)

USA
BRA
ESP


## 2.3.2 튜플 언패킹

튜플의 각 항목을 새로운 변수에 할당하는 것을 튜플 언패킹이라고 한다. 위의 예제에서 `city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)`와 같은 부분을 이야기한다. 또한 다음과 같이 함수의 인수로 튜플을 입력할 때, 인수 앞에 `*`를 붙여 튜플을 언패킹할 수 있다.

In [59]:
divmod(20,8)

(2, 4)

In [63]:
t = (20,8)
divmod(t) # 튜플을 바로 인수로 입력하면 에러 발생

TypeError: divmod expected 2 arguments, got 1

In [61]:
divmod(*t)

(2, 4)

### 초과 항목을 잡기 위해 * 사용하기

In [66]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [67]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [68]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

In [69]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

In [70]:
a, *body, c = range(5)
a, body, c

(0, [1, 2, 3], 4)

In [71]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

## 2.3.4 명명된 튜플

`collections.namedtuple()` 함수는 필드명과 클래스명을 추가한 튜플의 서브클래스를 생성하는 함수로서, 디버깅할 때 유용하다. 필드명이 클래스에 저장되므로 `namedtuple()`로 생성한 객체는 튜플과 동일한 크기의 메모리만 사용한다.

In [72]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')

In [74]:
tokyo = City('Tokyo', 'JP', 36.933, (35.689, 139.691))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689, 139.691))

In [76]:
tokyo.population, tokyo[1]

(36.933, 'JP')

명명된 튜플은 튜플에서 상속받은 속성 외에 `_fields` 클래스 속성, `_make(iterable)` 클래스 메서드, `_asdict()` 객체 메서드 등을 가지고 있다.

In [77]:
City._fields

('name', 'country', 'population', 'coordinates')

In [78]:
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613, 77.208))

In [81]:
delhi = City._make(delhi_data) # City(*delhi_data) 와 동일한 기능
delhi

City(name='Delhi NCR', country='IN', population=21.935, coordinates=LatLong(lat=28.613, long=77.208))

In [85]:
delhi._asdict() # OrderedDict 객체로 변환

OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', 21.935),
             ('coordinates', LatLong(lat=28.613, long=77.208))])

In [86]:
for key, value in delhi._asdict().items():
    print(key + ':', value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613, long=77.208)


## 2.3.5 불변 리스트로서의 튜플

리스트랑 튜플을 지원하는 메서드 리스트 확인

# 2.4 슬라이싱

## 2.4.1 슬라이스와 범위 지정시에 마지막 항목이 포함되지 않는 이유

* 세 개의 항목을 생성하는 `range(3)`나 `my_list[:3]`처럼 중단점만 이용할 때 길이를 계산하기 쉽다.
* 시작점과 중단점을 모두 지정할 때도 길이를 계산하기 쉽다. (중단점 - 시작점)
* x 인덱스를 기준으로 겹침 없이 시퀀스를 분할하기 쉽다. `my_list[:x]`와 `my_list[x:]`로 지정하면 된다.

## 2.4.2 슬라이스 객체

슬라이싱 작동 과정에서 `seq[start:stop:step]` 표현식을 평가하기 위해 파이썬은 `seq.__getitem__(slice(start, stop, step))`을 호출한다. 슬라이스 객체는 다음과 같이 슬라이스에 이름을 붙일 수 있게 해준다.

In [93]:
invoice = """
0.....6..............20.....
1909  Pimoroni      $17.50
1489  6mm Tactile    $4.95
"""

SKU = slice(0, 6)
DESCRIPTION = slice(6,20)
UNIT_PRICE = slice(20, None)

line_items = invoice.split('\n')[2:]
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

$17.50 Pimoroni      
 $4.95 6mm Tactile   
 


## 2.4.4 슬라이스에 할당하기

할당문의 왼쪽에 슬라이스 표기법을 사용하거나 del 문의 대상 객체로 지정함으로써 가변 시퀀스를 연결하거나, 잘라 내거나, 값을 변경할 수 있다.

In [102]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [103]:
l[2:5] = [20,30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [104]:
del l[5:7]
l

[0, 1, 20, 30, 5, 8, 9]

In [105]:
l[3::2] = [11,22]
l

[0, 1, 20, 11, 5, 22, 9]

In [106]:
l[2:5] = 100 # 오류 발생

TypeError: can only assign an iterable

In [107]:
l[2:5] = [100]
l

[0, 1, 100, 22, 9]

# 2.5 시퀀스에 덧셈과 곱셈 연산자 사용하기

하나의 시퀀스를 여러번 연결하려면 정수를 곱해서 표현한다. 이때 새로운 시퀀스가 만들어진다.

In [108]:
l = [1,2,3]
l*5, 5*l

([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3],
 [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3])

In [109]:
5 * 'abcd'

'abcdabcdabcdabcdabcd'

## 2.5.1 리스트의 리스트 만들기

내포된 리스트를 가진 리스트를 초기화할 때는 지능형 리스트를 사용하는 것이 가장 좋다.

In [111]:
board = [['_'] * 3 for _ in range(3)]
board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [112]:
board[1][2] = 'X'
board

[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]

이와 동일하게 작동하는 코드를 다음과 같이 짜면 안된다.

In [113]:
weird_board = [['_']*3]*3
weird_board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [115]:
weird_board[1][2] = 'X'
weird_board

[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]

이는 본질적으로 이 코드가 다음과 같이 작동하기 때문이다.

In [118]:
row = ['_'] * 3
board = []
for i  in range(3):
    board.append(row)
board[1][2] = 'X'
board

[['_', '_', 'X'], ['_', '_', 'X'], ['_', '_', 'X']]

In [119]:
row

['_', '_', 'X']

# 2.6 시퀀스의 복합 할당

`+=`과 `*=` 등의 복합 할당 연산자는 첫 번째 피연산자에 따라 상당히 다르게 작동한다. `+=` 연산자가 작동하도록 만드는 특수 메서드는 `__iadd__()` 다. 그런데 `__iadd__()` 메서드가 구현되어 있지 않으면 파이썬은 대신 `__add__()` 메서드를 호출한다. 다음 식을 생각해보자.

> a += b

a가 list, bytearray, array.array 등 가변 시퀀스인 경우 a의 값이 변경된다. 그런데 a가 `__iadd__()` 메서드를 구현하지 않은 경우 a += b 표현식은 a = a + b 가 되어 a 에 a+b 를 할당한다. 즉, `__iadd__()` 메서드의 구현 여부에 따라 a 변수가 가리키는 객체의 정체성이 바뀔 수도 있고 아닐 수도 있다.

In [123]:
l = [1,2,3]
l, id(l)

([1, 2, 3], 140193837450120)

In [124]:
l *= 2
l, id(l)

([1, 2, 3, 1, 2, 3], 140193837450120)

In [125]:
t = (1,2,3)
t, id(t)

((1, 2, 3), 140194119728440)

In [127]:
t *= 2 
t, id(t) # 튜플은 객체이 변경이 되지 않으므로 새로운 객체가 생성됨

((1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3), 140194132054232)

## 2.6.1 += 복합 할당 퀴즈

다음 코드를 실행하면 어떻게 될까?

> t = (1, 2, [30, 40]) <br>
> t[2] += [50, 60]

이 경우, 오류가 발생하지만 그럼에도 t 의 값은 (1, 2, [30, 40, 50, 60]) 으로 변경된다. 

# 2.7 list.sort() 와 sorted() 내장 함수

list.sort() 메서드는 리스트 내부를 변경해서 정렬하며, 그 결과로 None을 반환한다. random.shuffle() 함수도 비슷하게 작동한다. 이와 반대로 sorted() 함수는 새로운 함수를 생성해서 반환한다. 따라서 sorted() 함수는 불변 시퀀스 및 제너레이터를 포함해서 반복 가능한 모든 객체를 인수로 받을 수 있다(반환 값은 항상 리스트).

두 함수는 다음 두 키워드를 선택적으로 받는다.

* reverse(=False)
* key : 정렬에 사용할 키를 생성하기 위해 각 항목에 적용할 함수를 인수로 받는다. (eg. str.lower, len 등)

In [139]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits), fruits

(['apple', 'banana', 'grape', 'raspberry'],
 ['grape', 'raspberry', 'apple', 'banana'])

In [140]:
sorted(fruits, reverse=True)

['raspberry', 'grape', 'banana', 'apple']

In [141]:
sorted(fruits, key=len)

['grape', 'apple', 'banana', 'raspberry']

In [142]:
sorted(fruits, key=len, reverse=True)

['raspberry', 'banana', 'grape', 'apple']

In [143]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [144]:
fruits.sort()
fruits

['apple', 'banana', 'grape', 'raspberry']

# 2.8 정렬된 시퀀스를 bisect로 관리하기

bisect 모듈은 bisect()와 insort() 함수를 제공한다. bisect()는 이진 검색 알고리즘을 이용해서 시퀀스를 검색하고, insort()는 정렬된 시퀀스 안에 항목을 삽입한다.

## 2.8.1 bisect()로 검색하기

bisect(haystack, needle)은 정렬된 시퀀스(!)인 haystack 안에서 오름차순 정렬 상태를 유지한 채로 needle을 추가할 수 있는 위치를 찾아낸다. 즉, 해당 위치 앞에는 needle보다 같거나 작은 항목이 온다.

In [145]:
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)
        offset = position * ' |'
        print(ROW_FMT.format(needle, position, offset))

In [148]:
print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
demo(bisect.bisect)

haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14     | | | | | | | | | | | | | |31
30 @ 14     | | | | | | | | | | | | | |30
29 @ 13     | | | | | | | | | | | | |29
23 @ 11     | | | | | | | | | | |23
22 @  9     | | | | | | | | |22
10 @  5     | | | | |10
 8 @  5     | | | | |8 
 5 @  3     | | |5 
 2 @  1     |2 
 1 @  1     |1 
 0 @  0    0 


bisect의 행동은 두 가지 방식으로 조절할 수 있다.
* 선택 인수인 lo(=0)와 hi(len())를 이용하여 시퀀스 영역의 [lo:hi] 만 슬라이스하여 검색할 수 있다.
* bisect_left() 를 쓰면 기존 항목 바로 앞에 삽입한다.

bisect를 사용하면 수치형 값으로 테이블을 참조할 수 있으므로, 다음과 같이 시험 점수를 등급 문자로 변환할 수 있다.

In [149]:
def grade(score, breakpoints=[60,70,80,90], grades='FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]

In [151]:
[grade(score) for score in [33,99,77,70,89,90,100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

In [152]:
def grade_left(score, breakpoints=[60,70,80,90], grades='FDCBA'):
    i = bisect.bisect_left(breakpoints, score)
    return grades[i]

In [153]:
[grade_left(score) for score in [33,99,77,70,89,90,100]]

['F', 'A', 'C', 'D', 'B', 'B', 'A']

## bisect.insort()로 삽입하기

insort(seq, item)은 seq를 오름차순으로 유지한 채로 item을 seq에 삽입한다. bisect와 유사하게 lo, hi를 인수로 입력할 수 있으며, insort_left() 또한 존재한다.

In [154]:
import random

SIZE = 7
random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


# 2.9 리스트가 답이 아닐 때

리스트는 사용하기 편하지만 세부 요구사항에 따라 더 나은 자료형도 있다. 예를 들어 실수를 천만 개 저장해야 할 때는 배열이 훨씬 더 효율적이다. 한편 리스트의 양쪽 끝에 항목을 계속 추가하거나 삭제면서 FIFO나 LIFO 데이터 구조를 표현할 때는 덱(deque)가 더 빠르다. 또한 포함 여부를 판단해야할 때는 set형으로 구현하는 것이 좋다.

## 2.9.1 배열

리스트 안에 숫자만 들어 있다면 배열(array.array)이 리스트보다 훨씬 더 효율적이다. 배열은 가변 시퀀스가 제공하는 모든 연산을 지원하며, 빠르게 파일에 저장하고 읽어올 수 있는 frombytes()와 tofile() 메서드도 추가로 제공한다.

배열을 생성할 때는 배열에 저장되는 각 항목의 C 기반 형을 결정하는 문자인 타입코드(typecode)를 지정한다. 예를 들어 signed char에 대한 타입코드는 b이다. array('b') 배열을 생성하면 각 항목은 하나의 바이트로 저장되고 -128 에서 127까지의 정수로 해석된다.

In [169]:
for i in range(128):
    print("{:4} : {}".format(i, chr(i)))

   0 :  
   1 : 
   2 : 
   3 : 
   4 : 
   5 : 
   6 : 
   7 : 
   8 : 
   9 : 	
  10 : 

  11 : 
  12 : 
  13 : 
  14 : 
  15 : 
  16 : 
  17 : 
  18 : 
  19 : 
  20 : 
  21 : 
  22 : 
  23 : 
  24 : 
  25 : 
  26 : 
  27 : 
  28 : 
  29 : 
  30 : 
  31 : 
  32 :  
  33 : !
  34 : "
  35 : #
  36 : $
  37 : %
  38 : &
  39 : '
  40 : (
  41 : )
  42 : *
  43 : +
  44 : ,
  45 : -
  46 : .
  47 : /
  48 : 0
  49 : 1
  50 : 2
  51 : 3
  52 : 4
  53 : 5
  54 : 6
  55 : 7
  56 : 8
  57 : 9
  58 : :
  59 : ;
  60 : <
  61 : =
  62 : >
  63 : ?
  64 : @
  65 : A
  66 : B
  67 : C
  68 : D
  69 : E
  70 : F
  71 : G
  72 : H
  73 : I
  74 : J
  75 : K
  76 : L
  77 : M
  78 : N
  79 : O
  80 : P
  81 : Q
  82 : R
  83 : S
  84 : T
  85 : U
  86 : V
  87 : W
  88 : X
  89 : Y
  90 : Z
  91 : [
  92 : \
  93 : ]
  94 : ^
  95 : _
  96 : `
  97 : a
  98 : b
  99 : c
 100 : d
 101 : e
 102 : f
 103 : g
 104 : h
 105 : i
 106 : j
 107 : k
 108 : l
 109 : m
 110 : n
 1

천만 개의 무작위 실수를 가진 배열을 생성하고, 저장하고, 로딩하는 예제이다.

In [171]:
from array import array
from random import random

In [179]:
%%time
floats = array('d', (random() for _ in range(10**7)))

CPU times: user 834 ms, sys: 16 ms, total: 850 ms
Wall time: 849 ms


In [180]:
floats[-1]

0.1288579230853678

In [181]:
with open('floats.bin', 'wb') as f:
    floats.tofile(f)

In [182]:
floats2 = array('d')
with open('floats.bin', 'rb') as f:
    floats2.fromfile(f, 10**7)
    
floats2[-1]

0.1288579230853678

In [183]:
floats == floats2

True

## 2.9.2 메모리 뷰

메모리 뷰(memoryview) 내장 클래스는 공유 메모리 시퀀스형으로서 bytes를 복사하지 않고 배열의 슬라이스를 다룰 수 있게 해준다.

다음 예제는 16비트 정수 배열에서 바이트 하나를 변경하는 방법을 보여준다.

In [184]:
import array
numbers = array.array('h', [-2,-1,0,1,2])
memv = memoryview(numbers)

In [186]:
memv

In [189]:
len(memv), memv[0]

(5, -2)

In [190]:
memv_oct = memv.cast('B')
memv_oct.tolist()

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

In [191]:
memv_oct[5] = 4

In [192]:
numbers

array('h', [-2, -1, 1024, 1, 2])

## 2.9.4 덱 및 기타 큐

덱(collections.deque) 클래스는 큐의 양쪽 어디에서든 빠르게 삽입 및 삭제할 수 있도록 설계된 양방향 큐다. 최대 길이를 설정해서 제한된 항목만 유지할 수도 있으므로 덱이 꽉 찬 후에서는 새로운 항목을 추가할 때 반대쪽 항목을 버린다.

In [202]:
from collections import deque
dq = deque(range(10), maxlen=10)
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [203]:
dq.rotate(3)
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [204]:
dq.rotate(-3)
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [205]:
dq.appendleft(-1) # 요소 하나를 추가
dq

deque([-1, 0, 1, 2, 3, 4, 5, 6, 7, 8])

In [206]:
dq.extend([11,22,33]) # 반복가능한 요소를 추가
dq

deque([2, 3, 4, 5, 6, 7, 8, 11, 22, 33])

In [207]:
dq.extendleft([10,20,30,40])
dq

deque([40, 30, 20, 10, 2, 3, 4, 5, 6, 7])